In [1]:
import requests
import time
import threading
import logging

In [2]:
gists = ['https://gist.github.com/recluze/1d2989c7e345c8c3c542', 
        'https://gist.github.com/recluze/a98aa1804884ca3b3ad3', 
        'https://gist.github.com/recluze/5051735efe3fc189b90d', 
        'https://gist.github.com/recluze/460157afc6a7492555bb', 
        'https://gist.github.com/recluze/5051735efe3fc189b90d', 
        'https://gist.github.com/recluze/c9bc4130af995c36176d']

In [3]:
def thread_function(name):
    logging.info("Thread %s: starting", name)
    time.sleep(2)
    logging.info("Thread %s: finishing", name)

In [4]:
log_format = '%(asctime)s: %(message)s'
logging.basicConfig(format=log_format, level=logging.INFO,datefmt="%H:%M:%S")
threads = []
for index in range(3):
    logging.info("Main    : create and start thread %d.", index)
    th = threading.Thread(target = thread_function,args=(index,), daemon=True)
    threads.append(th)
    th.start()

for index, thread in enumerate(threads):
    logging.info("Main    : before joining thread %d.", index)
    thread.join()
    logging.info("Main    : thread %d done", index)

23:22:58: Main    : create and start thread 0.
23:22:58: Thread 0: starting
23:22:58: Main    : create and start thread 1.
23:22:58: Thread 1: starting
23:22:58: Main    : create and start thread 2.
23:22:58: Thread 2: starting
23:22:58: Main    : before joining thread 0.
23:23:00: Thread 0: finishing
23:23:00: Main    : thread 0 done
23:23:00: Main    : before joining thread 1.
23:23:00: Thread 1: finishing
23:23:00: Main    : thread 1 done
23:23:00: Main    : before joining thread 2.
23:23:00: Thread 2: finishing
23:23:00: Main    : thread 2 done


In [5]:
import concurrent.futures

In [6]:
with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    executor.map(thread_function,range(3))

23:23:00: Thread 0: starting
23:23:00: Thread 1: starting
23:23:00: Thread 2: starting
23:23:02: Thread 0: finishing
23:23:02: Thread 1: finishing
23:23:02: Thread 2: finishing


####  Race Conditions

In [7]:
logging.getLogger().setLevel(logging.DEBUG)
class FakeDatabase(object):
    def __init__(self):
        self.db_value  = 0
        self._lock = threading.Lock()
        
    def update(self):
        logging.info("Thread %s: starting update", name)
        next_value = self.db_value
        next_value +=1
        time.sleep(0.1)
        self.db_value += next_value
        logging.info("Thread %s: finishing update", name)

    def locked_update(self,name):
        logging.info("Thread %s: starting update of locked_update", name)
        logging.info("Thread %s about to lock", name)

        with self._lock:
            next_value = self.db_value
            next_value +=1
            time.sleep(0.1)
            self.db_value += next_value
            logging.info("Thread %s about to release lock", name)
        logging.debug("Thread %s after release", name)
        logging.info("Thread %s: finishing update", name)

In [8]:

fk_db = FakeDatabase()

In [9]:
logging.info("Testing update. Starting value is %d.", fk_db.db_value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(fk_db.update, index)
logging.info("Testing update. Ending value is %d.", fk_db.db_value)

23:23:02: Testing update. Starting value is 0.
23:23:02: Testing update. Ending value is 0.


In [10]:
fk_db2 = FakeDatabase()

In [11]:
logging.info("Testing update. Starting value is %d.", fk_db2.db_value)
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    for index in range(2):
        executor.submit(fk_db2.locked_update, index)
logging.info("Testing update. Ending value is %d.", fk_db2.db_value)

23:23:02: Testing update. Starting value is 0.
23:23:02: Thread 0: starting update of locked_update
23:23:02: Thread 0 about to lock
23:23:02: Thread 1: starting update of locked_update
23:23:02: Thread 1 about to lock
23:23:03: Thread 0 about to release lock
23:23:03: Thread 0 after release
23:23:03: Thread 0: finishing update
23:23:03: Thread 1 about to release lock
23:23:03: Thread 1 after release
23:23:03: Thread 1: finishing update
23:23:03: Testing update. Ending value is 3.


##### Deadlocks

A common problem when using Locks. As you saw, if the Lock has already been acquired, a second call to .acquire() will wait until the thread that is holding the Lock calls .release().
What do you think happens when you run this code:

In [1]:
import threading

#l = threading.Lock()
print("before first acquire")
#l.acquire()
print("before second acquire")
#l.acquire()
print("acquired lock twice")

before first acquire
before second acquire
acquired lock twice


When the program calls l.acquire() the second time, it hangs waiting for the Lock to be released. 
In this example, you can fix the deadlock by removing the second call, but deadlocks usually happen from one of two subtle things:

An implementation bug where a Lock is not released properly
A design issue where a utility function needs to be called by functions that might or might not already have the Lock

The first situation happens sometimes, but using a Lock as a context manager greatly reduces how often. It is recommended to write code whenever possible to make use of context managers, as they help to avoid situations where an exception skips you over the .release() call.

The design issue can be a bit trickier in some languages. Thankfully, Python threading has a second object, called RLock, that is designed for just this situation. It allows a thread to .acquire() an RLock multiple times before it calls .release(). That thread is still required to call .release() the same number of times it called .acquire(), but it should be doing that anyway.

##### Lock and RLock are two of the basic tools used in threaded programming to prevent race conditions. 